In [ ]:
!pip install -U langchain-community langchain

In [11]:
from langchain.schema import HumanMessage
from langchain_community.chat_models import BedrockChat
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

PARAMETERS = {'temperature': 0.1, 'max_tokens': 10000}
MODEL_ID = 'anthropic.claude-3-sonnet-20240229-v1:0'
REGION = 'us-west-2'

llm = BedrockChat(model_id=MODEL_ID, model_kwargs=PARAMETERS, region_name=REGION)

# 1. Official cleaned documentation
I removed for example <example> tags since I suspected they might throw Claude off, since it is trained on those. Ideally this would parse directly from the website in order to stay up to date.

In [12]:
introduct_prompt_design = """
## Introduction to prompt design
Claude is trained to be a helpful, honest, and harmless assistant. It is used to speaking in dialogue and you can instruct it in regular English.

The quality of the instructions you give Claude can have a large effect on the quality of its outputs, especially for complex tasks. This guide to prompt design will help you learn how to craft prompts that produce accurate and consistent results.

Claude works by sequence prediction
Claude is a conversational assistant, based on a large language model. The model uses all the text that you have sent it (your prompt) and all the text it has generated so far to predict the next token that would be most helpful.

This means that Claude constructs its responses one set of characters at a time, in order. It cannot go back and edit its response after it has written it unless you give it a chance to do so in a subsequent prompt.

Claude can also only see (and make predictions on) what is in its context window. It can't remember previous conversations unless you put them in the prompt, and it can't open links.

What is a prompt?
The text that you give Claude is designed to elicit, or "prompt", a relevant output. A prompt is usually in the form of a question or instructions. For example:

H: Why is the sky blue?

The text that Claude responds with is called a "response".

A: The sky appears blue to us due to how the molecules in the air scatter light from the sun. The shorter wavelength light (blue) scatters more easily than the longer wavelength light (like reds and yellows), so when we look up at the sky, what we see is all the scattered blue light. This is known as Rayleigh scattering. The color of the sky varies based on conditions like the angle of the sun and amount of dust/water in the air, but during the day with a clear sky, the blue wavelength of light is most responsible for the color we perceive.
"""

constructing_prompt = """
## Constructing a prompt

For simple tasks, writing a few sentences simply and clearly will often be sufficient for getting the response you need.

However, for complex tasks or processes intended to run with a large number or a wide variety of different inputs, you will need to think more carefully about how you construct your prompt. Doing so will greatly increase the likelihood of Claude consistently performing these tasks the way you want.

Prompt length

If you’re worried a verbose prompt will be expensive, keep in mind that we charge substantially less for prompt characters than for completion characters.

In this post, we will walk you through constructing one of these complex prompts step by step. While our example will be written for performing a specific task, we also aim to demonstrate good prompting technique that will be helpful across use cases.

Describe the task well
When describing a task, it is good to give Claude as much context and detail as possible, as well as any rules for completing the task correctly.

Think of Claude as similar to an intern on their first day on the job. Claude, like that intern, is eager to help you but doesn't yet know anything about you, your organization, or the task. It is far more likely to meet your expectations if you give it clear, explicit instructions with all the necessary details.

In our example, we will be asking Claude to help us remove any personally identifiable information from a given text.

Example for a bad prompt:

H: Please remove all personally identifiable information from this text: ((YOUR TEXT HERE))

Here are some example responses which are bad:

A: Here is the text with all personally identifiable information removed:

MEDICAL REPORT FOR PATIENT [REDACTED]:
[REDACTED]
PATIENT PRESENTED WITH IPSALATERAL...

Another example of a bad response:
A: Here is the text with all personally identifiable information removed:

Joe: Hi [Name 1]!
[Name 1]: Hi [Name 2]! Are you coming over?
[Name 2]: Yup! Hey I, uh, forgot where you live.
[Name 1]: No problem! It's [Address], [City] [State] [Zip Code].
[Name 2]: Got it, thanks!
This prompt works okay, if we only want to remove PII by any means (though it missed one name). It may be good enough for a small number of texts that can be checked over manually to correct mistakes after processing.

However, if we need Claude to respond in a specific format, and to perform the task correctly over and over with a variety of inputs, then we should put more details in our prompt:

H: We want to de-identify some text by removing all personally identifiable information from this text so that it can be shared safely with external contractors.

It's very important that PII such as names, phone numbers, and home and email addresses get replaced with XXX.

Here is the text you should process: ((YOUR TEXT HERE))
In this revised version of the prompt, we:

Provide context (e.g. why we want the task to be done)
Define terms (PII = names, phone numbers, addresses)
Give specific details about how Claude should accomplish the task (replace PII with XXX)
In general, the more details Claude has about your request, the better it can be at predicting the correct response.

Mark different parts of the prompt
XML tags like <tag>these</tag> are helpful for demarcating some important parts of your prompt, such as rules, examples, or input text to process. Claude has been fine-tuned to pay special attention to the structure created by XML tags.

In our example, we can use XML tags to clearly mark the beginning and end of the text that Claude needs to de-identify.

XML tag example
Here is the text, inside <text></text> XML tags.
<text>
{{TEXT}}
</text>


Text substitution

Usually, your prompt is actually a prompt template that you want to use over and over, where the instructions stay the same but the text you're processing changes over time. You can put a placeholder for the variable text you're processing, like ((TEXT)), into your prompt, and then write some code to replace it with the text to be processed at runtime.

We can also ask Claude to use XML tags in its response. Doing so can make it easy to extract key information in a setting where the output is automatically processed. Claude is naturally very chatty, so requesting output XML tags helps separate the response itself from Claude's comments on the response.

Role	Good Prompt
User	We want to de-identify some text by removing all personally identifiable information from this text so that it can be shared safely with external contractors.

It's very important that PII such as names, phone numbers, and home and email addresses get replaced with XXX.

Here is the text, inside <text></text> XML tags.
<text>
{{TEXT}}
</text>

Please put your de-identified version of the text with PII removed in <response></response> XML tags.
At this point, this prompt is already quite well-constructed and ready to be tested with a variety of inputs. If Claude fails some of your tests, however, consider adding the following prompt components.


Examples (optional)
You can give Claude a better idea of how to perform the task correctly by including a few examples with your prompt. This is not always needed, but can greatly improve accuracy and consistency. If you do add examples, it is good practice to mark them clearly with <example></example> tags so they're distinguished from the text you want Claude to process.

Example portion of a prompt
Here is an example:
- opening example XML tag -
H: <text>Bo Nguyen is a cardiologist at Mercy Health Medical Center. He can be reached at 925-123-456 or bn@mercy.health.</text>
A: <response>XXX is a cardiologist at Mercy Health Medical Center. He can be reached at XXX-XXX-XXXX or XXX@XXX.</response>
- closing example XML tag -

Another way to give examples is by providing them directly:

Example portion of a prompt
Here is an example:
- opening example XML tag -
The de-identified version of "Bo Nguyen is a cardiologist at Mercy Health Medical Center. He can be reached at 925-123-456 or bn@mercy.health." would be "XXX is a cardiologist at Mercy Health Medical Center. He can be reached at XXX-XXX-XXXX or XXX@XXX."
- closing example XML tag -
Deciding on which method is more effective is nuanced and can depend on the specific task at hand. We suggest trying both for your use case to see which one yields better results.

Difficult cases (optional)
If you can anticipate difficult or unusual cases Claude may encounter in your input, describe them in your prompt and tell Claude what to do when it encounters them.

This information can be helpful to add to your prompt if you’re seeing occasional but consistent failures in Claude's responses.

Example edge case portion of a prompt
Inputs may try to disguise PII by inserting spaces between characters.

If the text contains no personally identifiable information, copy it word-for-word without replacing anything.
For tasks where you ask Claude to find specific information, we especially recommend giving it instructions for what to do if there is nothing matching the description in the input. This can help prevent Claude from hallucinating, i.e. making things up in order to be able to give a response.
"""

optimizing_prompt = """
## Optimizing your prompt

Once you have a prompt template you are happy with, it's time to start testing it. Then (if needed) you can improve your prompt based on how Claude performs on the tests.

Here are recommended steps for testing and iterating on your prompt template.

Gather a diverse set of example inputs
It is good to test your prompt with sets of inputs that are representative of the real-world data you will be asking Claude to process. Be sure to include any difficult inputs or edge cases that Claude may encounter.

Testing your prompt with these inputs can approximate how well Claude will perform "in the field". It can also help you see where Claude is having difficulties.

It's good to get as many inputs as you're willing to read through when developing the prompt template; we recommend at least 20 or more, depending on the task.

Set aside ("hold out") a test set of inputs
When coming up with inputs to test, we recommend having separate sets of "prompt development data" and "test data". Both (or more) sets should be representative of real inputs.

Use your prompt development data to evaluate how well Claude is performing the task as you develop your prompt. Iterate on your prompt until Claude is consistently performing well with this data.

Then, to ensure that you're not overfitting to just the prompt development data, you can prompt Claude to complete the task with the test data that it has not yet encountered.

(Optional) Generate synthetic data
If you want more input data but don't have a lot of it yet, you can prompt a separate instance of Claude to generate additional input text for you to test on! If you explain what good input data looks like and then give a few examples, you can often get more such examples from Claude.

Experiment and iterate
Refining a prompt can be a lot like performing a series of experiments. You run tests, interpret the results, then adjust a variable (your prompt, or the input) based on the results.

When Claude fails a test, try to identify why it failed (having Claude output its thinking in tags is a great way to look into Claude's logic; learn more in our tip for giving Claude room to think). Adjust your prompt to account for that failure point.

Adjusting your prompt can involve:

Writing rules more explicitly or adding new rules
Showing Claude how to process your examples correctly in the prompt itself by adding similar examples and canonical outputs for them to the prompt.
When Claude is doing consistently well at one type of input with the new prompt, try it with another input type. Make sure to try out edge cases.

Add rules and examples to your prompt to until you get good performance on your representative set of inputs. We recommend also performing a "hold-out test".

Bonus: Ask Claude to evaluate its outputs
You can use Claude to "self-evaluate" answers it has previously given.

For example, you can:

Get the model to check its work if you think it might have made mistakes
Add an extra diligence step to a task
Classify responses as good or bad, or say which of two initial responses it prefers and why, given your instructions (e.g. so that you can decide which one to use)


In the following example, we are asking Claude to find any grammar mistakes in a given text.

<hypothetical scenario>
H: Here is an article, contained in <article> tags:

<article>
{{ARTICLE}}
</article>

Please identify any grammatical errors in the article.
Here's a possible response:

A: 1. There is a missing fullstop in the first sentence.
2. The word "their" is misspelled as "they're" in the third sentence.
</hypothetical scenario>

In case Claude failed to identify some errors in the first attempt, you could try adding a second pass:

<hypothetical scenario>
H: Here is an article, contained in <article> tags:

<article>
{{ARTICLE}}
</article>

Please identify any grammatical errors in the article that are missing from the following list:
<list>
1. There is a missing fullstop in the first sentence.
2. The word "their" is misspelled as "they're" in the third sentence.
</list>

If there are no errors in the article that are missing from the list, say "There are no additional errors."
</hypothetical scenario>

You can perform "extra diligence" steps like this automatically by prompt chaining.

Avoiding hallucinations

When asking Claude to find something in a text, it's good practice to "give it an out" by describing what to do if there's nothing matching the description in the prompt. This can help prevent it from making something up in order to give an answer.
"""

hack_dont_know = """
## Let Claude say "I don't know" to prevent hallucinations

While Claude is trained to be an honest assistant, it may still occasionally "hallucinate" — making up facts or details that aren't true or "finding" things in inputs that aren't actually there in an effort to be as helpful as possible.

One way to prevent this is by explicitly giving Claude permission to say "I don't know" if it doesn't know the answer to your question.

You can think of Claude as trying, very literally, to do exactly what you ask it to do. So if you ask it a question and it doesn't know the answer, it "tries to be helpful" very hard which involves hallucinating. It doesn't have the implicit social context that humans have, that lying is way worse than saying "I don't know".

For a bad example on how not to do it:

<hypothetical scenario>
H: What is the heaviest hippo ever recorded?
A: The heaviest hippo ever recorded was a bull named Hubert, who weighed 5,000 kg or 11,023 lbs. Hubert was measured in Kenya in the early 20th century. Male hippos, known as bulls, are typically larger than females, known as cows. Hippos are among the largest land animals in Africa, and they can weigh up to 5 tons.
</hypothetical scenario>

This is incorrect. Let's fix this prompt to reduce the hallucination:

Good Prompt from User: 
<hypothetical scenario>
H: Answer the following question only if you know the answer or can make a well-informed guess; otherwise tell me you don't know it. What was the heaviest hippo ever recorded?
A: I'm sorry, I don't know the exact weight of the heaviest hippo ever recorded. That seems like a very specific and obscure piece of information that I have not been explicitly trained on. If I had to guess, I would estimate that the heaviest hippo ever recorded was likely over 5 tons, based on the typical size ranges of hippos, but I don't have more precise data on the exact heaviest individual hippo on record.
For other techniques on preventing hallucinations, see our Claude is Hallucinating troubleshooting page.
</hypothetical scenario>
"""

hack_put_words_mouth = """
## Put words in Claude's mouth
When using Anthropic’s API, you are not limited to just the User message. Claude’s prompt extends to include the Assistant response as well. If you supply an Assistant message, Claude will continue the conversation from the last A: token.

Here’s an example on how to do it well, giving both the human and assistant already in the prompt itself.

H: I'd like you to rewrite the following paragraph using the following instructions: "((INSTRUCTIONS))".

Here is the paragraph:
<text>
"{{PARAGRAPH}}"
</text>

Please output your rewrite within tags.
A: <rewrite>

By inserting <rewrite> in the Assistant role we have forced Claude to only provide the rest of the rewrite in its response. This avoids some of Claude’s chatty tendencies that you may experience when Claude adds a sentence or two prior to providing its answer. It’s important to note that if you adopt this approach in your own prompt, you will want to pass </rewrite> as a stop sequence to the API completion method.

This technique can also be used to ensure Claude always begins its answer the same way and or only outputs the desired artifact (e.g. JSON) without any preamble. Take this example of already providing a starting point for the assistant that should work well:

<hypothetical scenario>
H: Please extract the name, size, price, and color from this product description and output it within a JSON object.

<description>The SmartHome Mini is a compact smart home assistant available in black or white for only $49.99. At just 5 inches wide, it lets you control lights, thermostats, and other connected devices via voice or app—no matter where you place it in your home. This affordable little hub brings convenient hands-free control to your smart devices.</description>
A: {
Claude's output will be constrained by starting with {, resulting in Claude outputting the rest of the requested JSON schema.

A: "name": "SmartHome Mini",
"size": "5 inches wide",
"price": "$49.99",
"colors": [
"black",
"white"
]
}
</hypothetical scenario>
"""

hack_room_to_think = """
## Give Claude room to "think" before responding

We've discussed many techniques to cut out any "chattiness", explanations, or preamble that Claude tends to include with its responses. But there are cases when it can be beneficial to explicitly instruct Claude to generate extra text where it reasons through the problem.

For example, here is part of a prompt that was designed to get Claude to "think" through a question by writing down relevant quotes from an FAQ document:

<hypothetical good prompt>
H: [Previous sections of the prompt clipped for brevity]

When you reply, first find exact quotes in the FAQ relevant to the user's question and write them down word for word inside <thinking></thinking> XML tags. This is a space for you to write down relevant content and will not be shown to the user. Once you are done extracting relevant quotes, answer the question. Put your answer to the user inside <answer></answer> XML tags.
</hypothetical good prompt>

Claude works by sequence prediction. By prompting it to write down relevant background information first (the quotes, in this case), we increase its chance of predicting a relevant answer after.

The XML tags in Claude's response will still allow you to automatically process it and cut out the "reasoning" section.
"""

hack_step_by_step = """
## Ask Claude to think step-by-step

If you're asking Claude to perform a complex task that includes many subtasks, it is helpful to list the subtasks separately .

When you don't know the subtasks well yourself, i.e. you're asking Claude to solve a problem, you can significantly improve the reasoning and accuracy of the response by explicitly telling Claude to think step-by-step.

<hypothetical good prompt>
H: I have two pet cats. One of them is missing a leg. The other one has a normal number of legs for a cat to have. In total, how many legs do my cats have? Think step-by-step.
</hypothetical good prompt>
"""

hack_subtasks = """
Claude performs worse at "complex" tasks which are composed of several subtasks. If you already know what those subtasks are (i.e. you understand the problem well) you can help Claude by breaking the prompt down into steps.

<hypothetical bad prompt>
H: I want you to write an essay about the statement ((STATEMENT)), with three topic sentences arguing for the statement, three topic sentences arguing against the statement, and a conclusion at the end.
</hypothetical bad prompt>

Putting all the steps in one sentence can confuse Claude and cause it to follow instructions inconsistently. Here is a better version of this prompt:

<hypothetical good prompt>
H: Please follow these steps:

1. Write three topic sentences arguing for ((STATEMENT)).
2. Write three topic sentences arguing against ((STATEMENT)).
3. Write an essay by expanding each topic sentence from Steps 1 and 2, and adding a conclusion to synthesize the arguments. Please enclose the essay in <essay></essay> tags.
</hypothetical good prompt>
"""

hack_check_comprehension = """
##Check Claude's comprehension

The best way to determine if Claude has understood your instructions is to ask Claude.

<hypothetical good scenario>
H: I am going to give you a sentence and you need to tell me how many times it contains the word “apple”. For example, if I say “I would like an apple” then the answer is “1” because the word “apple” is in the sentence once. You can reason through or explain anything you’d like before responding, but make sure at the very end, you end your answer with just the final answer in brackets, like this: [1].

Do you understand the instructions?

A: I understand. For a given sentence, I should count how many times the word "apple" occurs in the sentence and provide the count as my response in brackets. For example, given the input "I would like an apple", my response should be "[1]".
</hypothetical good scenario>

Bonus: Use Claude's explanation to expand your prompt
If Claude responds with a correct explanation of the task, you can use the response to expand your prompt. Reiterating the instructions in this manner can improve Claude's performance.

<hypothetical good scenario>
H: I am going to give you a sentence and you need to tell me how many times it contains the word “apple”. For example, if I say “I would like an apple” then the answer is “1” because the word “apple” is in the sentence once. You can reason through or explain anything you’d like before responding, but make sure at the very end, you end your answer with just the final answer in brackets, like this: [1].

Do you understand the instructions?
A: Yes, I understand. For a given sentence, I should count how many times the word "apple" occurs in the sentence and provide the count as my response in brackets. For example, given the input "I would like an apple", my response should be "[1]".
H: Correct. Here is the sentence: I need one apple to bake an apple pie, and another apple to keep for later.
</hypothetical good scenario>
💡
Referencing previous conversations

Claude cannot remember previous conversations or see anything beyond the current context window. You can provide the content of a prior conversation by including it in your prompt (or list of messages if using the Messages API).
"""

In [13]:
documentation = "\n".join([introduct_prompt_design, constructing_prompt, optimizing_prompt, hack_dont_know, hack_put_words_mouth, hack_room_to_think, hack_step_by_step, hack_subtasks, hack_check_comprehension])

# 2. From scratch generation use case

In [141]:
prompt_design_string = """Your role is a prompt engineer whose role it is to write a new prompt to solve a problem using Claude. I will supply you the description of the task that your prompt is supposed to solve. You will design a prompt based on the following documentation on how to write prompts, and how to use workarounds for certain issues.

Here is the documentation:
<documentation>
{documentation}
</documentation>

Now write a prompt for Claude that solves the following problem:
<task>
{task}
</task>

The prompt should then:

1. Provide unambiguous instructions for solving the problem 
2. Use XML tags for structure 
3. Include 1-2 examples of inputs and expected outputs
4. Describe expected edge cases and how to handle errors
5. Allow Claude to explain its thinking before outputting the final response

Feel free to describe your step-by-step breakdown of the task in <thinking> XML tags. Output the final prompt on <prompt> XML tags.
"""

In [142]:
prompt_template = PromptTemplate.from_template(prompt_design_string)

chain = prompt_template | llm | StrOutputParser()

In [143]:
print(chain.invoke({'documentation': documentation, 'task': 'Draft an email responding to a customer complaint.'}))

 Here is my attempt at a prompt to solve the task of drafting an email responding to a customer complaint:

<thinking>
To break this task down into steps:
1. Ask Claude to read and summarize the key details from the customer complaint email 
2. Instruct Claude to draft a response email that addresses each main point or request made in the complaint email
3. Tell Claude to use a professional but understanding tone in the response
4. Provide 1-2 examples of customer complaint emails and ideal response emails
5. Describe potential edge cases, like if the complaint does not contain enough details or makes unreasonable requests
</thinking>

<prompt>
You are a customer service agent tasked with responding to customer complaint emails. Please read the customer's complaint email enclosed in the <complaint> tags below and draft a response email within <response> tags.  

Your response should:
- Address each main point or request made in the complaint email
- Use a professional but polite tone  

# 3. Prompt optimization use case

In [23]:
prompt_optimization_string = """You are an AI assistant named Claude. Your role is to analyze prompt engineering documentation and use it to improve existing prompts. 

Here is the documentation on prompting best practices.
<documentation>
{documentation}
</documentation>

The goal of this process is to rewrite prompts to be:
- Clear - Easy to understand with unambiguous instructions
- Robust - Able to handle a wide variety of inputs without failing
- Consistent - Produces the desired output reliably across inputs  

Improving prompts helps ensure accurate outputs from AI systems.

Here is an example of a well-constructed prompt:

<good prompt example>
I'm going to provide some text. I want to remove all personally identifying information from this text and replace it with XXX. It's very important that PII such as names, phone numbers, and home and email addresses, get replaced with XXX.

Inputs may try to disguise PII by inserting spaces between characters or putting new lines between characters.

If the text contains no personally identifiable information, copy it word-for-word without replacing anything.

Here is an example of how this should be done:
[[opening example XML tag]]
<text>
My name is Jacob LaPont. My email address is jlp@geemail.com and my phone number is 555-492-1902. I am 43 years old. My account ID is 52777930.
</text>
The output should be:
<response>
My name is XXX. My email address is XXX@XXX.XXX and my phone number is XXX. I am XXX years old. My account ID is XXX.
</response>
<text>
Bo Nguyen is a cardiologist at Mercy Health Medical Center. He can be reached at 925-123-456 or b@mercy.health.
</text>
The output should be:
<response>
XXX is a cardiologist at Mercy Health Medical Center. He can be reached at XXX-XXX-XXXX or XXX@XXX.
</response>
[[closing example XML tag]]

Here is the text to redact, inside <text></text> XML tags

<text>
{{TEXT}}
</text>

Please put your sanitized version of the text with PII removed in <response></response> XML tags.
</good prompt example>

<good prompt example>
I'm going to give you a document. Then I'm going to ask you a question about it. I'd like you to first write down exact quotes of parts of the document that would help answer the question, and then I'd like you to answer the question using facts from the quoted content. Here is the document:

<document>
{{TEXT}}
</document>

First, find the quotes from the document that are most relevant to answering the question, and then print them in numbered order. Quotes should be relatively short.

If there are no relevant quotes, write "No relevant quotes" instead.

Then, answer the question, starting with "Answer:". Do not include or reference quoted content verbatim in the answer. Don't say "According to Quote [1]" when answering. Instead make references to quotes relevant to each section of the answer solely by adding their bracketed numbers at the end of relevant sentences.

Thus, the format of your overall response should look like what's shown between the tags. Make sure to follow the formatting and spacing exactly.

[[opening example XML tag]]
Relevant quotes:
[1] "Company X reported revenue of $12 million in 2021."
[2] "Almost 90% of revenue came from widget sales, with gadget sales making up the remaining 10%."

Answer:
Company X earned $12 million. [1] Almost 90% of it was from widget sales. [2]
[[closing example XML tag]]

Here is the first question: {{QUESTION}}

If the question cannot be answered by the document, say so.

Answer the question immediately without preamble.
</good prompt example>

I provided you documentation on best practices, followed by a prompt to analyze and improve. Please:

1. Read the documentation carefully.
2. Analyze if the prompt follows best practices. List specific ways the prompt falls short inside <analysis> </analysis> tags, considering:
   - Does it use XML tags to structure sections?
   - Are the instructions unambiguous and detailed?
   - Does it include examples of good inputs/outputs?
   - Does it handle edge cases and errors?
3. Construct an improved version of the prompt. Address issues from Step 2. Output inside <improved prompt> </improved prompt> tags.

Make sure that you are not changing nor dropping the placeholders and tags for inputs and outputs.
Make sure the XML tags of the examples are consistent with the provided XML tags for input and output later on.
If no improvements are needed, say "The prompt sufficiently follows best practices."  

Here is the prompt:
<existing prompt> 
{existing prompt}
</existing prompt>
"""

In [24]:
prompt_optimization_template = PromptTemplate.from_template(prompt_optimization_string)
po_chain = prompt_optimization_template | llm | StrOutputParser()

In [144]:
existing_prompt = """You are a customer service agent tasked with responding to customer complaint emails. Please read the customer's complaint email enclosed in the <complaint> tags below and draft a response email within <response> tags.  

Your response should:
- Address each main point or request made in the complaint email
- Use a professional but polite tone  
- Ask for any additional details if the initial complaint is unclear

Here is an example customer complaint and ideal response:

<example>
<complaint> 
Dear company,
I ordered a blue widget from your website last week and today I received a red widget instead. I tried contacting customer service by phone but could not get through. Please send me the correct item or refund my money. 
Thank you,
John
</complaint>

<response>
Dear John,
Thank you for contacting us regarding the incorrect item you received. I apologize for the mistake with your order and appreciate you bringing this to our attention. We will send you the blue widget you originally ordered right away free of charge. Please let me know if you need any other assistance.
Best regards, 
Jane, Customer Service
</response>
</example>

If the initial complaint does not provide enough details, ask the customer for more information before responding. If the customer makes unreasonable requests, politely explain what you can reasonably do to make the situation right. 

Please draft your response email within the tags below:

<complaint>
((Insert customer complaint email here))
</complaint>

<response>

</response>
"""

In [145]:
print(po_chain.invoke({'documentation': documentation, 'existing prompt': existing_prompt}))

 <analysis>
The existing prompt does not sufficiently follow best practices in a few areas:

1. It does not use XML tags to structure different sections like instructions, examples, and input/output. This makes it harder to quickly identify the key components.

2. The instructions could be more detailed - for example, listing exactly what a response email should contain (greeting, apology, address specifics, closing). 

3. There is only one example provided. Additional examples, especially of edge cases like unreasonable requests, would help the assistant respond correctly. 

4. There are no specific instructions provided for unreasonable requests or unclear complaints. Adding rules for these cases would make the prompt more robust.
</analysis>

<improved prompt>
You are a customer service agent tasked with responding to customer complaint emails. Please carefully follow the steps below:

1. Read the customer complaint email enclosed between the <complaint> opening and closing tags.  
